In [1]:
import numpy as np
import pandas as pd

In [2]:
credit = pd.read_csv('credit_record.csv')
app = pd.read_csv('application_record.csv')
app.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
credit.loc[credit['STATUS'].isin(['C','X','0']), 'STATUS']=0 #GOOD
credit.loc[credit['STATUS'].isin(['5','4','3','2','1']), 'STATUS'] = 1 #BAD 
credit['STATUS'] = credit['STATUS'].astype(int)
credit.drop(['MONTHS_BALANCE'],axis = 1,inplace = True)

In [4]:
credit_id = credit.groupby(['ID']).agg(max).reset_index()
credit_id 

,ID,STATUS
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,0
45981,5150483,0
45982,5150484,0
45983,5150485,0


In [5]:
app_df = app.merge(credit_id, on = 'ID', how = 'inner')
#DAYS_BIRTH NEEDS TO BE RECALCULATED -> AGE IN YEARS
#DAYS_EMPLOYED NEED TO BE RECALCULATED -> YEARS EMPLOYED
app_df['AGE'] = app_df['DAYS_BIRTH'].abs()/365
app_df['AGE'] = app_df['AGE'].round(0)
app_df['AGE'] = app_df['AGE'].astype(int) 
app_df.drop(['DAYS_BIRTH'], axis = 1, inplace = True)

In [6]:
app_df.loc[app_df['DAYS_EMPLOYED'] > 0, 'DAYS_EMPLOYED'] = 0
app_df.loc[app_df['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED'] = app_df['DAYS_EMPLOYED'].abs()


In [7]:
app_df['YEARS_EMPLOYED'] = app_df['DAYS_EMPLOYED']/365 
app_df['YEARS_EMPLOYED'] = app_df['YEARS_EMPLOYED'].round(2)
app_df['YEARS_EMPLOYED'].describe()

count    36457.000000
mean         6.028365
std          6.484368
min          0.000000
25%          1.120000
50%          4.250000
75%          8.640000
max         43.050000
Name: YEARS_EMPLOYED, dtype: float64

In [8]:
app_df.drop(['DAYS_EMPLOYED'], axis = 1,inplace = True)

In [9]:
app_df.columns

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE',
       'CNT_FAM_MEMBERS', 'STATUS', 'AGE', 'YEARS_EMPLOYED'],
      dtype='object')

In [10]:
app_df = app_df[['ID', 'AGE', 'YEARS_EMPLOYED','CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'CNT_FAM_MEMBERS', 'STATUS']]
app_df

,ID,AGE,YEARS_EMPLOYED,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5008804,33,12.44,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,NaN,2.0,1
1,5008805,33,12.44,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,NaN,2.0,1
2,5008806,59,3.11,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,Security staff,2.0,0
3,5008808,52,8.36,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,0
4,5008809,52,8.36,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,48,6.63,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,Managers,2.0,1
36453,5149834,34,3.63,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,Medicine staff,2.0,1
36454,5149838,34,3.63,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,Medicine staff,2.0,1
36455,5150049,49,1.79,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,Sales staff,2.0,1


In [11]:
app_df.loc[app_df['OCCUPATION_TYPE'].isnull(), 'OCCUPATION_TYPE'] = 'MISSING OCCUPATION'
app_df.isnull().sum()

ID                     0
AGE                    0
YEARS_EMPLOYED         0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
STATUS                 0
dtype: int64

In [12]:
categorical = app_df[['CODE_GENDER', 'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']]
numerical = app_df[['AGE','YEARS_EMPLOYED','CNT_CHILDREN','AMT_INCOME_TOTAL','CNT_FAM_MEMBERS']]
label = app_df[['STATUS']]

In [13]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
cat_encde = categorical.apply(le.fit_transform)
X = pd.concat([cat_encde,numerical],axis = 1)

In [14]:
Y = label.copy()
Y['LABEL'] = np.nan
Y.loc[Y['STATUS'] == 0, 'LABEL'] = 'GOOD'
Y.loc[Y['STATUS'] == 1, 'LABEL'] = 'BAD'
labels = Y[['LABEL']]
labels

,LABEL
0,BAD
1,BAD
2,GOOD
3,GOOD
4,GOOD
...,...
36452,BAD
36453,BAD
36454,BAD
36455,BAD


In [15]:
labels['LABEL'].value_counts()

GOOD    32166
BAD      4291
Name: LABEL, dtype: int64

In [16]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest= train_test_split(X,labels.values.ravel(), test_size = 0.2, stratify = labels)

lr = LogisticRegression()
lr.fit(xtrain,ytrain)
yhat = lr.predict(xtest)
score = lr.score(xtest,ytest)
print("The score with Logistic Regression is: ", round(score, 2))


The score with Logistic Regression is:  0.88


In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
print(classification_report(ytest, yhat))

              precision    recall  f1-score   support

         BAD       0.00      0.00      0.00       858
        GOOD       0.88      1.00      0.94      6434

    accuracy                           0.88      7292
   macro avg       0.44      0.50      0.47      7292
weighted avg       0.78      0.88      0.83      7292



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SMOTE

In [19]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 'minority', random_state = 42)
X_resampled, labels_resampled = sm.fit_sample(X, label)
X_resampled = pd.DataFrame(data = X_resampled, columns = xtrain.columns)
labels_resampled = pd.DataFrame(data = labels_resampled, columns = ['STATUS'])

In [20]:
sm_df = X_resampled

sm_df['STATUS'] = labels_resampled['STATUS']

sm_df

,CODE_GENDER,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,AGE,YEARS_EMPLOYED,CNT_CHILDREN,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,STATUS
0,1,1,1,1,4,1,0,4,10,33,12.440000,0,427500.0,2.0,1
1,1,1,1,1,4,1,0,4,10,33,12.440000,0,427500.0,2.0,1
2,1,1,1,1,4,4,1,1,17,59,3.110000,0,112500.0,2.0,0
3,0,0,0,1,0,4,3,1,15,52,8.360000,0,270000.0,1.0,0
4,0,0,0,1,0,4,3,1,15,52,8.360000,0,270000.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64327,0,0,0,0,0,1,1,1,15,30,2.850000,1,675000.0,3.0,1
64328,0,0,0,1,4,4,1,1,10,36,13.056934,2,72000.0,4.0,1
64329,0,0,0,1,1,4,1,1,10,61,0.000000,0,225000.0,2.0,1
64330,1,1,1,1,1,4,1,1,10,64,0.000000,0,112500.0,2.0,1


In [21]:
labels_resampled.value_counts()

STATUS
0         32166
1         32166
dtype: int64

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
xtrain, xtest, ytrain, ytest = train_test_split(sm_df,sm_df['STATUS'], test_size = 0.2, random_state = 42)
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(xtrain,ytrain)
yhat = knn.predict(xtest)
score = knn.score(xtest,ytest)
print("The score with KNN is: ", round(score, 2))

The score with KNN is:  0.94


In [24]:
print(classification_report(ytest, yhat))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      6388
           1       0.93      0.96      0.94      6479

    accuracy                           0.94     12867
   macro avg       0.94      0.94      0.94     12867
weighted avg       0.94      0.94      0.94     12867

